In [1]:
!pip install "pandas==1.0.0"


In [2]:
!pip install -U pandas-profiling

Requirement already up-to-date: pandas-profiling in /Users/giovannibruner/opt/anaconda3/lib/python3.8/site-packages (2.13.0)
  Using cached pandas-1.2.4-cp38-cp38-macosx_10_9_x86_64.whl (10.5 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.0
    Uninstalling pandas-1.0.0:
      Successfully uninstalled pandas-1.0.0


In [3]:
!pip install awswrangler


In [118]:

import awswrangler as wr
import pandas as pd
import numpy as np
import os

# Get Data

### 1) Tracciato Sezioni

In [9]:
%%time

tracciato_sezioni = wr.athena.read_sql_query("""SELECT * FROM 
                         gimi.sezioni_istat_tracciato
                         """, 
                         database="gimi",
                         ctas_approach = False
                        )

CPU times: user 470 ms, sys: 98.7 ms, total: 568 ms
Wall time: 2.43 s


In [15]:
for i in tracciato_sezioni.itertuples():
    print(i.variabile, i.definizione)
    

NOME_CAMPO DEFINIZIONE
CODREG Codice numerico che identifica univocamente la regione nellambito del territorio nazionale
REGIONE Denominazione della regione
CODPRO Codice numerico che identifica univocamente la provincia nellambito del territorio nazionale
PROVINCIA Denominazione della provincia
CODCOM Codice numerico che identifica univocamente il comune nellambito del territorio provinciale
COMUNE Denominazione del comune
PROCOM Codice numerico che identifica univocamente il comune nellambito del territorio nazionale. Il valore è ottenuto dalla concatenazione del campo CODPRO con il campo CODCOM a tre digit
SEZ2011 Codice numerico che identifica univocamente la sezione di censimento 2011 nellambito del territorio nazionale. Il valore è ottenuto dalla concatenazione del campo PROCOM con il campo NSEZ a 7 digit
NSEZ "Numero che identifica univocamente la sezione di censimento 2011 nellambito del territorio comunale. Con riferimento alle zone in contestazione
ACE Numero che identi

In [ ]:
['REGIONE', 'PROVINCIA', 'COMUNE', 'NSEZ', 'P1', 'P47']

In [132]:
%%time

sql_query = """
    
    select REGIONE as regione,
           PROVINCIA as provincia,
           COMUNE as comune,
           NSEZ as sezione,
           P1 as popolazione_totale,
           P47 as popolazione_laureata,
           P14 as popolazione_0_4_anni,
           centoid_lat as centroid_long,
           centoid_long as centroid_lat
    from gimi.sezioni_istat

"""

tempdf = wr.athena.read_sql_query(
                         sql_query, 
                         database="gimi",
                         ctas_approach = False
                        )

CPU times: user 2.31 s, sys: 584 ms, total: 2.89 s
Wall time: 20.5 s


In [133]:
tempdf.head()

,regione,provincia,comune,sezione,popolazione_totale,popolazione_laureata,popolazione_0_4_anni,centroid_long,centroid_lat
0,Sicilia,Catania,Misterbianco,483,0,0,0,15.031943,37.538692
1,Sicilia,Catania,Misterbianco,491,5,3,0,14.973666,37.475860
2,Sicilia,Catania,Misterbianco,496,0,0,0,15.009686,37.487184
3,Sicilia,Catania,Misterbianco,507,0,0,0,14.998557,37.509253
4,Sicilia,Catania,Misterbianco,508,195,13,25,14.999774,37.525221


In [134]:
# Remove sections with less people than the median of the population distribution
caliper = np.percentile(tempdf['popolazione_totale'], 50)
tempdf = tempdf[tempdf['popolazione_totale'] > caliper].copy()

In [138]:
# Calculate proportions of people wiht a degree on total

tempdf['degree_prop'] = (tempdf['popolazione_laureata'] + 1e-5
                        ) / (tempdf['popolazione_totale'] + 1e-5)

In [139]:
# Calculate proportions of people wiht a degree on total

tempdf['0_4_anni_prop'] = (tempdf['popolazione_0_4_anni'] + 1e-5
                        ) / (tempdf['popolazione_totale'] + 1e-5)

In [140]:
# Create target score (tot_populazion * degree_prop)
tempdf['target_score'] = tempdf['degree_prop'] * (tempdf['popolazione_totale'] * 0.5)

In [141]:
# Create target score (tot_populazion * degree_prop)
tempdf['target_score_childs'] = tempdf['degree_prop'] * (tempdf['popolazione_0_4_anni'] * 0.5)

In [142]:
%%time

# Create string from openstreetmap

search_strings = []

for r in tempdf.itertuples():
    
    lat = r.centroid_lat
    lon = r.centroid_long
    
    # Create open street map search strings
    main_path = "https://www.openstreetmap.org"
    query = f"search?query={lat}%2C%{lon}#map=18/{lat}/{lon}"
    search_string = os.path.join(main_path, query)
    search_strings.append(search_string)
    
tempdf['os_search_string'] = search_strings

CPU times: user 3.02 s, sys: 172 ms, total: 3.19 s
Wall time: 3.38 s


In [160]:
# For each region find the top 100 sections - target_score

regions_target_score = []

n = 100

for reg in tempdf['regione'].unique():
    
    tt = tempdf[tempdf['regione'] == reg]
    topn = tt.sort_values(by = 'target_score', ascending = False).head(n)
    
    regions_target_score.append(topn)
    
target_score_df = pd.concat(regions_target_score)

In [165]:
# For each region find the top 100 sections - target_score_child

regions_target_score = []

n = 100

for reg in tempdf['regione'].unique():
    
    tt = tempdf[tempdf['regione'] == reg]
    topn = tt.sort_values(by = 'target_score', ascending = False).head(n)
    
    regions_target_score.append(topn)
    
target_score_child_df = pd.concat(regions_target_score)

In [167]:
target_score_df.to_excel('target_score_geo.xlsx', index = False)

In [168]:
target_score_df.columns

Index(['regione', 'provincia', 'comune', 'sezione', 'popolazione_totale',
       'popolazione_laureata', 'popolazione_0_4_anni', 'centroid_long',
       'centroid_lat', '0_4_anni_prop', 'degree_prop', 'target_score',
       'target_score_childs', 'os_search_string'],
      dtype='object')

In [151]:
tempdf[tempdf['regione'] == 'Campania'
      ].sort_values(by = 'target_score', ascending = False).head(30)

,regione,provincia,comune,sezione,popolazione_totale,popolazione_laureata,popolazione_0_4_anni,centroid_long,centroid_lat,0_4_anni_prop,degree_prop,target_score,target_score_childs,os_search_string
335326,Campania,Napoli,Napoli,6421061,3052,855,121,14.227780,40.856635,0.039646,0.280144,427.500004,16.948722,https://www.openstreetmap.org/search?query=40....
278272,Campania,Caserta,Aversa,59,2962,673,192,14.199929,40.963930,0.064821,0.227211,336.500004,21.812289,https://www.openstreetmap.org/search?query=40....
335463,Campania,Napoli,Napoli,6522451,1636,615,64,14.208614,40.820043,0.03912,0.375917,307.500003,12.02934,https://www.openstreetmap.org/search?query=40....
335386,Campania,Napoli,Napoli,6421681,2248,584,79,14.220208,40.858973,0.035142,0.259786,292.000004,10.261566,https://www.openstreetmap.org/search?query=40....
335264,Campania,Napoli,Napoli,6320331,1398,523,67,14.223005,40.840647,0.047926,0.374106,261.500003,12.532547,https://www.openstreetmap.org/search?query=40....
335322,Campania,Napoli,Napoli,6421021,2166,492,103,14.226981,40.861273,0.047553,0.227147,246.000004,11.698061,https://www.openstreetmap.org/search?query=40....
335504,Campania,Napoli,Napoli,6522871,1229,475,59,14.196187,40.805436,0.048007,0.386493,237.500003,11.401546,https://www.openstreetmap.org/search?query=40....
335269,Campania,Napoli,Napoli,6320391,1113,401,47,14.219060,40.839809,0.042228,0.360288,200.500003,8.466757,https://www.openstreetmap.org/search?query=40....
335331,Campania,Napoli,Napoli,6421111,1642,390,53,14.228512,40.852850,0.032278,0.237515,195.000004,6.294154,https://www.openstreetmap.org/search?query=40....
335387,Campania,Napoli,Napoli,6421691,1218,387,40,14.222976,40.858705,0.032841,0.317734,193.500003,6.35468,https://www.openstreetmap.org/search?query=40....


In [130]:
tempdf[tempdf['regione'] == 'Lazio'].sort_values(by = 'target_score', ascending = False).iloc[0]['os_search_string']

'https://www.openstreetmap.org/search?query=41.89570180901009%2C%12.47981616053679#map=18/41.89570180901009/12.47981616053679'